In [1]:
!pip install torch 
!pip install peft
!pip install bitsandbytes
!pip install transformers
!pip install trl 
!pip install accelerate
!pip install einops


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer
from langchain.document_loaders import HuggingFaceDatasetLoader

In [3]:
base_model = "microsoft/Phi-3-mini-128k-instruct"
#base_model = "microsoft/phi-2"
#new_model = "phi-3-matt-medicine-election"
new_model = "phi-3-Taiwan-election"


# Dataset
dataset = load_dataset("wenlianghuang/election_in_Taiwan",split="train")

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "right"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# Quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.int16,
    bnb_4bit_use_double_quant=False,
)

# Load base moodel
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    trust_remote_code=True,
    #attn_implementation="flash_attention_2",
    device_map={"": 0}
)

model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

In [5]:
peft_config = LoraConfig(
    r= 64,          
    lora_alpha= 16,
    lora_dropout=0.05, #0.1
    bias="none",
    task_type="CAUSAL_LM",
    #target_modules= ["Wqkv", "out_proj"],
    target_modules="all-linear",
)

In [6]:
# Set training arguments
training_arguments = TrainingArguments(
    output_dir = "./results_0701",
    num_train_epochs = 1,
    fp16 = False,
    bf16 = False,
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    gradient_accumulation_steps = 1,
    gradient_checkpointing = True,
    max_grad_norm = 0.3,
    learning_rate = 2e-4,
    weight_decay = 0.001,
    optim = "paged_adamw_32bit",
    lr_scheduler_type = "cosine",
    max_steps = -1,
    warmup_ratio = 0.03,
    group_by_length = True,
    save_steps = 0,
    logging_steps = 25,
)

In [7]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length= None,
    tokenizer=tokenizer,
    args=training_arguments,
)

# Train model
trainer.train()

c:\Users\Wen-Liang\AppData\Local\Programs\Python\Python311\Lib\site-packages\trl\trainer\sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


  0%|          | 0/29 [00:00<?, ?it/s]

c:\Users\Wen-Liang\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.float16.


{'loss': 1.1879, 'grad_norm': 0.10392235964536667, 'learning_rate': 0.0001975746552556772, 'epoch': 0.1}
{'loss': 0.9435, 'grad_norm': 0.125300794839859, 'learning_rate': 0.00018550053929480202, 'epoch': 0.2}
{'loss': 0.8871, 'grad_norm': 0.11743128299713135, 'learning_rate': 0.00016449948488669639, 'epoch': 0.3}
{'loss': 0.8349, 'grad_norm': 0.17206382751464844, 'learning_rate': 0.000136764169663272, 'epoch': 0.4}
{'loss': 0.8924, 'grad_norm': 0.11997736245393753, 'learning_rate': 0.00010519038181318999, 'epoch': 0.5}
{'loss': 0.8035, 'grad_norm': 0.1924498975276947, 'learning_rate': 7.307467669163655e-05, 'epoch': 0.6}
{'loss': 0.8315, 'grad_norm': 0.16196095943450928, 'learning_rate': 4.377019014049223e-05, 'epoch': 0.7}
{'loss': 0.8125, 'grad_norm': 0.1959877759218216, 'learning_rate': 2.03365443542764e-05, 'epoch': 0.8}
{'loss': 0.857, 'grad_norm': 0.13457253575325012, 'learning_rate': 5.22039891260262e-06, 'epoch': 0.9}
{'loss': 0.768, 'grad_norm': 0.15213711559772491, 'learning_

TrainOutput(global_step=250, training_loss=0.881824562072754, metrics={'train_runtime': 1940.2631, 'train_samples_per_second': 0.515, 'train_steps_per_second': 0.129, 'total_flos': 1.996136183660544e+16, 'train_loss': 0.881824562072754, 'epoch': 1.0})

In [ ]:
# Save trained model
trainer.model.save_pretrained(new_model)

In [ ]:


prompt = "Who is the current president in Taiwan?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=300,)
result = pipe(f"### Instruction: {prompt}")
print(result[0]['generated_text'])

In [11]:
# Reload model and merge it with LoRA parameters
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    cache_dir="",
    device_map={"": 0},
)
model = PeftModel.from_pretrained(model, new_model)
model = model.merge_and_unload()

config.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

In [12]:
# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [15]:
model.push_to_hub(new_model, use_temp_dir=False)

In [16]:
tokenizer.push_to_hub(new_model, use_temp_dir=False)

HfHubHTTPError:  (Request ID: Root=1-667e7bea-30df35e50b2701bf75082658;de829395-ecc5-48cc-8ec5-614a6c3ce843)

403 Forbidden: You don't have the rights to create a model under the namespace "wenlianghuang".
Cannot access content at: https://huggingface.co/api/repos/create.
If you are trying to create or update content,make sure you have a token with the `write` role.